In [ ]:
# Load imports and configuration

%load_ext autoreload
%autoreload 2

import hydra
from hydra.core.global_hydra import GlobalHydra

GlobalHydra.instance().clear()
hydra.initialize(version_base=None, config_path="../conf")
cfg = hydra.compose(config_name="config")

tickers = cfg.market.tickers
interval = cfg.market.interval
fee_rate = cfg.market.fee_rate
initial_cash = cfg.market.initial_cash
risk_free_rate = cfg.market.risk_free_rate_annual

from skopt.space import Integer, Real

from modules.performance.strategy import Strategy
from modules.data_services.data_loaders import load_data
from modules.performance.statistical_tests import engle_granger_cointegration
from modules.data_services.data_utils import load_btc_benchmark, save_to_parquet
from modules.visualization.plots import plot_pnl, plot_positions, plot_zscore
from modules.performance.stats import calculate_stats
from modules.core.models import StrategyResult

In [ ]:
pair_selection_start = "2024-01-01"
pair_selection_end = "2024-03-01"

In [ ]:
# Load data and perform pair selection

ps_df = load_data(
    tickers=tickers,
    start=pair_selection_start,
    end=pair_selection_end,
    interval=interval
)

eg_df = engle_granger_cointegration(
    df=ps_df,
    source="log_prices"
)

eg_df.head(5)

In [ ]:
# Select TOP5 pairs

tickers = eg_df.iloc[0:5, 0].tolist()
print(f"Selected pairs: {tickers}")

In [ ]:
opt_pre_start = "2024-01-01"
opt_start = "2024-02-01"
opt_end = "2024-03-01"

test_beta_calculation_start = "2024-01-01"
test_pre_start = "2024-02-01"
test_start = "2024-03-01"
test_end = "2024-04-01"

In [ ]:
# Configure optimization spaces

static_params = {
    "stop_loss": 10  # Disable SL
}

param_space = [
    Integer(10, 400, name="rolling_window"),
    Real(1.01, 4.00, name="entry_threshold"),
    Real(0.0, 1.00, name="exit_threshold"),
    # Real(1.01, 3.00, name="stop_loss"),
]

metric = ("sortino_annual_with_trades", "net")

In [ ]:
# Perform multi-pair simulation

results = []
for ticker in tickers:
    ticker_x = ticker.split('-')[0]
    ticker_y = ticker.split('-')[1]
    print(f"Pair: {ticker_x}/{ticker_y}")

    # Set up a Strategy
    bt = Strategy(
        ticker_x=ticker_x,
        ticker_y=ticker_y,
        start=test_beta_calculation_start,
        end=test_end,
        interval=interval,
        fee_rate=fee_rate,
        initial_cash=initial_cash,
        risk_free_rate_annual=risk_free_rate,
        beta_hedge="static_hedge",
        source="log",
    )

    # Run optimization
    best_params, best_score = bt.run_optimization(
        opt_start=opt_start,
        opt_end=opt_end,
        static_params=static_params,
        param_space=param_space,
        metric = metric,
    )
    print(best_params, best_score)

    entry_threshold = best_params["entry_threshold"]
    exit_threshold = best_params["exit_threshold"]
    # stop_loss = best_params["stop_loss"]
    stop_loss = 10
    rolling_window = best_params["rolling_window"]

    # Run strategy
    result = bt.run_strategy(
        rolling_window=rolling_window,
        entry_threshold=entry_threshold,
        exit_threshold=exit_threshold,
        stop_loss=stop_loss,
        test_start=test_start,
        test_end=test_end,
    )
    save_to_parquet(
        df=result.data,
        file_name=f"result_data_{ticker_x}_{ticker_y}",
    )

    # Show data
    print(result.stats)

    results.append(result)

    plot_positions(result, directory="top5_test", save=True, show=True)
    btc_data = load_btc_benchmark(
        test_start=test_start,
        test_end=test_end,
        interval=interval,
    )
    plot_pnl(result, btc_data, directory="top5_test", save=True, show=True)
    plot_zscore(result, directory="top5_test", save=True, show=True)

In [ ]:
# Calculate summary

cols_to_sum = ['position', 'total_return', 'total_fees', 'net_return']
summary_data = results[0].data[cols_to_sum].copy()
for p in results[1:]:
    summary_data += p.data[cols_to_sum]

summary_data['total_return_pct'] = summary_data.data['total_return'] / (initial_cash * 5)
summary_data['net_return_pct'] = summary_data.data['net_return'] / (initial_cash * 5)
summary_data['position'] = summary_data.data['position'] / 5

summary_stats = calculate_stats(
    df=summary_data,
    initial_cash=initial_cash,
    interval=interval,
    risk_free_rate_annual=risk_free_rate,
)

summary = StrategyResult(
    data=summary_data,
    ticker_x=...,
    ticker_y=...,
    start=...,
    end=...,
    interval=interval,
    fee_rate=fee_rate,
    stats=summary_stats,
)

In [ ]:
# Show statistics

summary.stats

In [ ]:
# Visualization

from modules.visualization.plots import plot_pnl

plot_positions(summary, show=True, save=False)

btc_data = load_data(
    tickers=['BTCUSDT'],
    start=test_start,
    end=test_end,
    interval=interval,
)
btc_data['BTC_return'] = btc_data['BTCUSDT'].pct_change()
btc_data.loc[btc_data.index[0], 'BTC_return'] = 0.0
btc_data['BTC_cum_return'] = (1 + btc_data['BTC_return']).cumprod() - 1

plot_pnl(summary, btc_data, show=True, save=False)